In [1]:
import os
os.chdir('..')
os.getcwd()


'/Users/wliao0504/code/clif/pyCLIF'

In [2]:
import yaml

with open('config/config.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [ ]:
import pandas as pd
from pathlib import Path

# Import individual table classes
from clifpy.tables.patient import Patient
from clifpy.tables.hospitalization import Hospitalization  
from clifpy.tables.vitals import Vitals
from importlib import reload

In [4]:
# Initialize the tables
patient = Patient.from_file(
    data_directory=config['tables_path'],
    filetype='parquet', 
    timezone='UTC'
)
hospitalization = Hospitalization.from_file(
    data_directory=config['tables_path'], 
    filetype='parquet', 
    timezone='UTC'
)

Loading clif_patient.parquet
Data loaded successfully from clif_patient.parquet
death_dttm: null count before conversion= 85
death_dttm: Your timezone is UTC, Converting to your site timezone (UTC).
death_dttm: null count after conversion= 85
Validation completed with 2 error(s). See `errors` attribute.
Loading clif_hospitalization.parquet
Data loaded successfully from clif_hospitalization.parquet
admission_dttm: null count before conversion= 0
admission_dttm: Your timezone is UTC, Converting to your site timezone (UTC).
admission_dttm: null count after conversion= 0
discharge_dttm: null count before conversion= 0
discharge_dttm: Your timezone is UTC, Converting to your site timezone (UTC).
discharge_dttm: null count after conversion= 0
Validation completed successfully for table hospitalization.


In [ ]:
%load_ext autoreload
%autoreload 2

from clifpy.tables.medication_admin_continuous import MedicationAdminContinuous

mac = MedicationAdminContinuous.from_file(
    data_directory=config['tables_path'], 
    filetype='parquet', 
    timezone='UTC'
)

vitals = Vitals.from_file(
    data_directory=config['tables_path'], 
    filetype='parquet', 
    timezone='UTC'
)
vitals_df = vitals.df

Loading clif_medication_admin_continuous.parquet
Data loaded successfully from clif_medication_admin_continuous.parquet
admin_dttm: null count before conversion= 0
admin_dttm: Your timezone is UTC, Converting to your site timezone (UTC).
admin_dttm: null count after conversion= 0
Validation completed with 4 error(s). See `errors` attribute.
Loading clif_vitals.parquet
Data loaded successfully from clif_vitals.parquet
recorded_dttm: null count before conversion= 0
recorded_dttm: Your timezone is UTC, Converting to your site timezone (UTC).
recorded_dttm: null count after conversion= 0
Validation completed with 1 error(s). See `errors` attribute.


In [6]:
mac.validate()

Validation completed with 4 error(s). See `errors` attribute.


In [7]:
mac.df

,hospitalization_id,med_order_id,admin_dttm,med_name,med_category,med_group,med_route_name,med_route_category,med_dose,med_dose_unit,mar_action_name,mar_action_category
0,20044587,8238199,2113-08-25 17:31:00+00:00,Dextrose 5%,dextrose,others,NaN,NaN,5.0,mL/hour,start,NaN
1,20044587,8238199,2113-08-27 21:26:00+00:00,Dextrose 5%,dextrose,others,NaN,NaN,0.0,mL/hour,Stopped,NaN
2,20044587,9731494,2113-08-25 21:43:00+00:00,Dextrose 5%,dextrose,others,NaN,NaN,200.0,mL/hour,start,NaN
3,20044587,9731494,2113-08-25 21:58:00+00:00,Dextrose 5%,dextrose,others,NaN,NaN,0.0,mL/hour,FinishedRunning,NaN
4,20044587,8890793,2113-08-25 22:03:00+00:00,Dextrose 5%,dextrose,others,NaN,NaN,100.0,mL/hour,start,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6805,29974575,4770561,2131-03-04 11:00:00+00:00,Magnesium Sulfate (Bolus),magnesium,others,NaN,NaN,50.0,mL/hour,start,NaN
6806,29974575,196330,2131-03-05 10:26:00+00:00,Magnesium Sulfate (Bolus),magnesium,others,NaN,NaN,50.0,mL/hour,start,NaN
6807,29974575,6947205,2131-03-08 03:30:00+00:00,Magnesium Sulfate (Bolus),magnesium,others,NaN,NaN,50.0,mL/hour,start,NaN
6808,29974575,6582775,2131-03-01 16:25:00+00:00,Magnesium Sulfate (OB-GYN),magnesium,others,NaN,NaN,2.0,grams/hour,start,NaN


In [8]:
df = mac.standardize_dose_unit(vitals_df, mac.df)
df.head()

,hospitalization_id,med_order_id,admin_dttm,med_name,med_category,med_group,med_route_name,med_route_category,med_dose,med_dose_unit,...,mar_action_category,weight_kg,weight_recorded_dttm,rn,med_dose_unit_lower,time_multiplier,pt_weight_adjustment,dose_multiplier,med_dose_converted,med_dose_unit_converted
0,20044587,5689218,2113-08-25 11:30:00-06:00,Propofol,propofol,sedation,NaN,NaN,40.064104,mcg/kg/min,...,NaN,77.8,2113-08-25 09:07:00-06:00,1,mcg/kg/min,1.000000,77.8,1.0,3116.987284,mcg/min
1,20044587,8238199,2113-08-25 11:31:00-06:00,Dextrose 5%,dextrose,others,NaN,NaN,5.000000,mL/hour,...,NaN,77.8,2113-08-25 09:07:00-06:00,1,ml/hour,0.016667,1.0,1.0,0.083333,ml/min
2,20044587,5689218,2113-08-25 15:28:00-06:00,Propofol,propofol,sedation,NaN,NaN,59.891909,mcg/kg/min,...,NaN,77.8,2113-08-25 09:07:00-06:00,1,mcg/kg/min,1.000000,77.8,1.0,4659.590548,mcg/min
3,20044587,9731494,2113-08-25 15:43:00-06:00,Dextrose 5%,dextrose,others,NaN,NaN,200.000000,mL/hour,...,NaN,77.8,2113-08-25 09:07:00-06:00,1,ml/hour,0.016667,1.0,1.0,3.333333,ml/min
4,20044587,9731494,2113-08-25 15:58:00-06:00,Dextrose 5%,dextrose,others,NaN,NaN,0.000000,mL/hour,...,NaN,77.8,2113-08-25 09:07:00-06:00,1,ml/hour,0.016667,1.0,1.0,0.000000,ml/min


In [9]:
mac_df = mac.df

In [10]:
mac._run_universal_validations()